In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv",header=0)
items=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv",header=0)
item_categories=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv",header=0)
shops=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv",header=0)
test=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv",header=0).set_index('ID')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
sns.distplot(train["item_price"], fit=norm)
plt.title("item_price Distplot", color = "darkred")
plt.show()

seems item prices are skewed. few items have very high price.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
sns.distplot(train["item_cnt_day"], fit=norm)
plt.title("item_cnt_day Distplot", color = "darkred")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(train["item_price"], train["item_cnt_day"], color = 'red')
plt.xlabel('item_price')
plt.ylabel('item_cnt_day')
plt.show()

we can see outliers

In [ ]:
print(train[train["item_price"]>100000].count)

single record. so we can remove them

In [ ]:
print(train[train["item_cnt_day"]>900].count)

only one record above 1000. so we can remove them

In [ ]:
train.describe()

In [ ]:
max_date_block_num=max(train["date_block_num"])
print(max_date_block_num)
print(train[train["item_price"]<0].count)

price is negative for 1 item so removing it.

In [ ]:
train=train[(train["item_price"]>0) & (train["item_price"]<100000) & (train["item_cnt_day"]<=1000)]

In [ ]:
count = train.groupby(['shop_id','item_id'], as_index = False).item_price.nunique()
count[count.item_price >1]

In [ ]:
train[(train.shop_id==59) & (train.item_id==22164)]

different prices for same product in same shop with diff month. may be some sale or seasonal offerings. Good to add month as column

In [ ]:
import re 
train['month'] = train.date.map(lambda x : re.search('\.(.*?)\.', x).group(1)).astype(np.int64)

#dataset['month']

In [ ]:
test['date_block_num']=max_date_block_num+1
cols = ['date_block_num','shop_id','item_id']
dataset=pd.concat([train, test], ignore_index=True, sort=False, keys=cols)
dataset.fillna(0, inplace=True)
dataset['item_sale_revenue'] = dataset['item_price']*dataset['item_cnt_day']
shop_monthly_sale = dataset.groupby(['shop_id','date_block_num'], as_index = False).agg(shop_monthly_sale=('item_cnt_day','sum'), shop_monthly_revenue=('item_sale_revenue','sum')).sort_values(by=['shop_id','date_block_num'], ascending=True)
shop_monthly_sale["shop_monthly_sale_prev1"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_sale'].shift(1)
shop_monthly_sale["shop_monthly_sale_prev2"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_sale_prev1'].shift(1)
shop_monthly_sale["shop_monthly_sale_prev3"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_sale_prev2'].shift(1)
shop_monthly_sale["shop_monthly_sale_prev4"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_sale_prev3'].shift(1)
shop_monthly_sale["shop_monthly_sale_prev5"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_sale_prev4'].shift(1)
shop_monthly_sale["shop_monthly_sale_prev6"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_sale_prev5'].shift(1)
shop_monthly_sale["shop_monthly_revenue_prev1"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_revenue'].shift(1)
shop_monthly_sale["shop_monthly_revenue_prev2"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_revenue_prev1'].shift(1)
shop_monthly_sale["shop_monthly_revenue_prev3"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_revenue_prev2'].shift(1)
shop_monthly_sale["shop_monthly_revenue_prev4"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_revenue_prev3'].shift(1)
shop_monthly_sale["shop_monthly_revenue_prev5"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_revenue_prev4'].shift(1)
shop_monthly_sale["shop_monthly_revenue_prev6"]=shop_monthly_sale.groupby(['shop_id'])['shop_monthly_revenue_prev5'].shift(1)
shop_monthly_sale=shop_monthly_sale.fillna(0)

shop_monthly_item_sale = dataset.groupby(['shop_id','item_id','date_block_num'], as_index = False).agg(shop_monthly_item_sale=('item_cnt_day','sum'), shop_monthly_item_revenue=('item_sale_revenue','sum')).sort_values(by=['shop_id','item_id','date_block_num'], ascending=True)
shop_monthly_item_sale["shop_monthly_item_sale_prev1"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_sale'].shift(1)
shop_monthly_item_sale["shop_monthly_item_sale_prev2"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_sale_prev1'].shift(1)
shop_monthly_item_sale["shop_monthly_item_sale_prev3"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_sale_prev2'].shift(1)
shop_monthly_item_sale["shop_monthly_item_sale_prev4"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_sale_prev3'].shift(1)
shop_monthly_item_sale["shop_monthly_item_sale_prev5"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_sale_prev4'].shift(1)
shop_monthly_item_sale["shop_monthly_item_sale_prev6"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_sale_prev5'].shift(1)

shop_monthly_item_sale["shop_monthly_item_revenue_prev1"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_sale'].shift(1)
shop_monthly_item_sale["shop_monthly_item_revenue_prev2"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_revenue_prev1'].shift(1)
shop_monthly_item_sale["shop_monthly_item_revenue_prev3"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_revenue_prev2'].shift(1)
shop_monthly_item_sale["shop_monthly_item_revenue_prev4"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_revenue_prev3'].shift(1)
shop_monthly_item_sale["shop_monthly_item_revenue_prev5"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_revenue_prev4'].shift(1)
shop_monthly_item_sale["shop_monthly_item_revenue_prev6"]=shop_monthly_item_sale.groupby(['shop_id','item_id'])['shop_monthly_item_revenue_prev5'].shift(1)
shop_monthly_item_sale=shop_monthly_item_sale.fillna(0)

In [ ]:
#print(shop_monthly_sale[shop_monthly_sale.shop_id==10])

In [ ]:
#preparing masterdataset
masterdata=pd.merge(dataset,shop_monthly_sale, on=['shop_id','date_block_num'], how='left')
masterdata=pd.merge(masterdata,shop_monthly_item_sale, on=['shop_id','item_id','date_block_num'], how='left')
masterdata.dtypes

In [ ]:
print(masterdata.count())
print(masterdata.isnull().sum())
type(masterdata)

In [ ]:
masterdata.drop(['date'], axis=1, inplace=True)
# Encoding categorical data
# Encoding the Independent Variable
'''from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
X=masterdata
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['date_block_num', 'shop_id', 'item_id', 'month'])], remainder='passthrough')
masterdata = ct.fit_transform(masterdata)'''

'''from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
masterdata = sc.fit_transform(masterdata)'''

X_train = masterdata[masterdata.date_block_num < 33].drop(['shop_monthly_item_sale'], axis=1)
y_train = masterdata[masterdata.date_block_num < 33]['shop_monthly_item_sale']
X_valid = masterdata[masterdata.date_block_num == 33].drop(['shop_monthly_item_sale'], axis=1)
y_valid = masterdata[masterdata.date_block_num == 33]['shop_monthly_item_sale']
X_test =  masterdata[masterdata.date_block_num == 34].drop(['shop_monthly_item_sale'], axis=1)

In [ ]:
# Training the Multiple Linear Regression model on the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred=regressor.predict(X_valid)
from sklearn import metrics
score_rf=metrics.r2_score(y_valid, y_pred)
print(score_rf)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

rf_param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = rf_param_grid, n_iter = 100, cv = 3, 
                               verbose=2, random_state=42, n_jobs = -1)
'''rf_random.fit(X_train, y_train)
y_pred=rf_random.predict(X_valid)
from sklearn import metrics
score_rf=metrics.r2_score(y_valid, y_pred)
print(score_rf)'''

In [ ]:
import xgboost as xgb

import xgboost as xgb

model = xgb.XGBRegressor(importance_type='gain', learning_rate=0.1)
model.fit(X_train, y_train,verbose=True)
 
score = model.score(X_train, y_train)   
print("Training score: ", score) 
score = model.score(X_valid, y_valid)   
print("validatio score: ", score) 


In [ ]:
Y_test = model.predict(X_test)
print(Y_test[0:5])
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})


In [ ]:
submission.to_csv('xgb_submission.csv', index=False)